In [ ]:
from BiblioMeter_GUI.Page_Classes import App_Test

app = App_Test()
app.mainloop()

In [ ]:
# Python program demonstrating Multiple selection
# in Listbox widget with a scrollbar
  
from tkinter import *
  
window = Tk()
window.title('Multiple selection')
  
# for scrolling vertically
yscrollbar = Scrollbar(window)
yscrollbar.pack(side = RIGHT, fill = Y)
  
label = Label(window,
              text = "Select the languages below :  ",
              font = ("Times New Roman", 10), 
              padx = 10, pady = 10)
label.pack()
list = Listbox(window, selectmode = "multiple", 
               yscrollcommand = yscrollbar.set)
  
# Widget expands horizontally and 
# vertically by assigning both to
# fill option
list.pack(padx = 10, pady = 10,
          expand = YES, fill = "both")
  
x =["C", "C++", "C#", "Java", "Python",
    "R", "Go", "Ruby", "JavaScript", "Swift",
    "SQL", "Perl", "XML"]
  
for each_item in range(len(x)):
      
    list.insert(END, x[each_item])
    list.itemconfig(each_item, bg = "lime")

list.selection_set(first = 5)

# Attach listbox to vertical scrollbar
yscrollbar.config(command = list.yview)
window.mainloop()

In [ ]:
# Standard library imports
import os
from pathlib import Path

# 3rd party imports
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd

# Local imports
import BiblioAnalysis_Utils as bau
from BiblioMeter_GUI.BiblioMeter_AllPagesFunctions import five_last_available_years
from BiblioMeter_GUI.BiblioMeter_UsefulClasses import ColumnFilter
from BiblioMeter_GUI.Globals_GUI import STOCKAGE_ARBORESCENCE
from BiblioMeter_GUI.Globals_GUI import SET_1
from BiblioMeter_GUI.Globals_GUI import SUBMIT_FILE_NAME


from BiblioAnalysis_Utils.BiblioSpecificGlobals import DIC_OUTDIR_PARSING
from BiblioAnalysis_Utils.BiblioSpecificGlobals import FOLDER_NAMES

bibliometer_path = "S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files"

newWindow = tk.Tk()
newWindow.title('Création du filtre')
newWindow.resizable(False, False)
newWindow.geometry(f"600x600+{newWindow.winfo_rootx()}+{newWindow.winfo_rooty()}")

# Useful alias
bdd_mensuelle_alias = STOCKAGE_ARBORESCENCE['general'][0]
submit_alias = SUBMIT_FILE_NAME

### Choose which year you want to be working with #############################################################################################################
years_list = five_last_available_years(bibliometer_path)
variable = tk.StringVar(newWindow)
variable.set(years_list[0])

    # Création de l'optionbutton des années
OptionButton = tk.OptionMenu(newWindow, variable, *years_list)
#OptionButton.place(anchor = 'center', relx = 0.25, rely = 0.15)

    # Création du label
Label = tk.Label(newWindow, text = '''Choisir l'année de travail :''')
#Label.place(anchor = 'center', relx = 0.10, rely = 0.15)
###############################################################################################################################################################

### Bouton qui va permettre de lancer la création du filtre ###################################################################################################
Button = tk.Button(newWindow, 
                   text = 'Création du filtre', 
                   command = lambda: _create_filter())
#Button.place(anchor = 'center', relx = 0.45, rely = 0.15)
################################################################################################################################################################

## Create a Frame
main_frame = tk.LabelFrame(newWindow, highlightbackground = 'red', highlightthickness = 1)
backup_frame = tk.LabelFrame(newWindow, highlightbackground = 'red', highlightthickness = 1)

# Create a canvas
my_canvas = tk.Canvas(main_frame)
my_canvas.pack(side = tk.LEFT, fill = tk.BOTH, expand = 1)

# Add a scrollbar to the canvas
my_scrollbar = ttk.Scrollbar(main_frame, orient = tk.VERTICAL, command = my_canvas.yview)
my_scrollbar.pack(side = tk.RIGHT, fill = tk.Y)

# Configure the canvas
my_canvas.configure(yscrollcommand = my_scrollbar.set)
my_canvas.bind('<Configure>', lambda e : my_canvas.configure(scrollregion = my_canvas.bbox("all")))

# Create another frame inside the canvas
second_frame = tk.Frame(my_canvas)
second_frame.pack()

main_frame.pack(fill = tk.BOTH, expand = 1, padx = 10, pady = 10)
backup_frame.pack(fill = tk.BOTH, expand = 1, padx = 10, pady = 10)

# Add that new frame to a window in the canvas
my_canvas.create_window((0,0), window = second_frame, anchor = 'nw')

### Path to the submit file ###################################################################################################################################
submit_path = Path(bibliometer_path) / Path(variable.get()) / Path(bdd_mensuelle_alias) / Path(submit_alias)
###############################################################################################################################################################

### Charger la df #############################################################################################################################################
df = pd.read_excel(submit_path)
df['Nom Prénom']=df.apply(lambda x:'%s %s' % (x['Nom'], x['Prénom']),axis=1)
df_1 = df[SET_1]
df_1.fillna('', inplace=True)
###############################################################################################################################################################

for i in range(len(df_1.columns)):
    tmp = ColumnFilter(second_frame, df_1.columns[i], df_1)
    tmp.place(y = i)

newWindow.mainloop()

In [ ]:
### Exo création de filtre
import pandas as pd

nom_nouveau_fichier = 'hello.xlsx'

df_submit = pd.read_excel(r'S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\111- Ludovic Desmeuzes\BiblioMeter_Files\2022\BDD multi mensuelle\submit.xlsx')

columns_to_keep = set(['Pub_id', 
                   'Idx_author', 
                   'Authors',  
                   'DOI', 
                   'ISSN', 
                   'LITEN_France',  
                   'Document_type',
                   'Matricule', 
                   'Nom', 
                   'Prénom', 
                   'Dpt/DOB (lib court)', 
                   'Service (lib court)', 
                   'Laboratoire (lib court)', 
                   'Laboratoire (lib long)',
                   'List_of_OTP',
                   'HOMONYM'])

columns_to_remove = list(set(df_submit.columns.tolist()) - columns_to_keep)

# Remettre les Pub_ID dans l'ordre
df_submit.sort_values(by=['Pub_id', 'Dpt/DOB (lib court)'], ascending=False, inplace = True)
df_submit.sort_values(by=['Pub_id', 'Idx_author'], ascending=True, inplace = True)

for column in columns_to_remove:
    df_submit.drop(column, 1, inplace = True)
df_submit.columns

new_df = pd.DataFrame(df_submit.columns)
new_df.to_excel(r'S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\111- Ludovic Desmeuzes\BiblioMeter_Files\Filtres\OTP_DTNM.xlsx')

In [ ]:
in_path = r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2021/BDD multi mensuelle/submit.xlsx'
out_path = r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2021/BDD multi mensuelle/par_dep.xlsx'

#def filtrer_par_departement(in_path, out_path, dep_to_keep, et_ou):
'''
Faire attention à ce que path mène à un fichier submit

Args : 
path : chemin vers fichier submit
dep_to_keep : liste de 0 et 1 disant quel département inclure

Returns :
un fichier excel
'''

# Local imports
from BiblioMeter_GUI.Globals_GUI import SET_OTP

# 3rd party imports
import pandas as pd

### Charger la df et ajouter les 4 colonnes ###################################################################################################################
df = pd.read_excel(in_path)
df_OTP = df[SET_OTP]
df_OTP.fillna('', inplace=True)
df_OTP.set_index('Pub_id', inplace = True)

data = [0] * len(df_OTP)
df_OTP['DTNM'] = data
df_OTP['DTS'] = data
df_OTP['DTCH'] = data
df_OTP['DEHT'] = data
###############################################################################################################################################################

for i in df_OTP.index.unique().to_list():

    if isinstance(df_OTP.loc[i], pd.Series):
        df_inter_pub_id = df_OTP.loc[i].to_frame().T
    else:
        df_inter_pub_id = df_OTP.loc[i]

    for j in df_inter_pub_id['Idx_author']:

        filtre_inter_author = df_inter_pub_id['Idx_author'] == j
        df_inter_inter = df_inter_pub_id[filtre_inter_author]

        if df_inter_inter['Dpt/DOB (lib court)'].to_list()[0] == 'DTNM':
            df_OTP.loc[i,'DTNM'] = 1
        elif df_inter_inter['Dpt/DOB (lib court)'].to_list()[0] == 'DTS':
            df_OTP.loc[i,'DTS'] = 1
        elif df_inter_inter['Dpt/DOB (lib court)'].to_list()[0] == 'DEHT':
            df_OTP.loc[i,'DEHT'] = 1
        else:
            df_OTP.loc[i,'DTCH'] = 1

#df_OTP.drop(['Idx_author'], axis=1, inplace = True)
df_OTP.reset_index(inplace = True)
df_OTP.drop_duplicates(subset = ['Pub_id'], inplace = True)
df_OTP.set_index('Pub_id', inplace = True)
df_OTP.to_excel(out_path)

In [ ]:
def replace_by_one(df, row, column):
    df.loc[row, column] = 1
    
df_OTP.loc[15].apply(replace_by_one(df, 15, 'DTCH'))

In [1]:
import pandas as pd
from BiblioMeter_FUNCTS.BiblioMeterFonctions import add_authors_name_list

in_path = r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2018/BDD multi mensuelle/submit.xlsx'


add_authors_name_list(r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2018/BDD multi mensuelle/submit.xlsx', 
                      r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2018/BDD multi mensuelle/submit.xlsx')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LD259969\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LD259969\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LD259969\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Number of detected displays: 2



Select Id of gui prime-display (value: 0 to 1; default:1) 


C:\Users\LD259969\Documents\PyVenv\BiblioMeter\BiblioMeter_FUNCTS\BiblioMeterFonctions.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter['Authors Fullname List'] = authors_fullname_list


In [ ]:
if isinstance(df, pd.DataFrame):
    to_check = ['Nom', 'Prénom']
    for i in to_check:
        if 'Nom' not in df:
            raise KeyError(f"The column {i} is not in DataFrame")
else:
    raise TypeError(f"The variable {df} is not of proper type, it has to be a DataFrame")
    
df['Nom Prénom'] = df['Nom'] + ', ' + df['Prénom']

In [ ]:
df_inter = df[(df['Pub_id'] == 3)]

In [ ]:
authors_fullname_list = '; '.join(df_inter['Nom Prénom'].tolist())

In [ ]:
authors_fullname_list

In [ ]:
df_inter_lol = pd.DataFrame.from_dict({x[0]:['; '.join(x[1]['Nom Prénom'])] for x in df.groupby(['Pub_id'])}).T

In [ ]:
df_out = pd.DataFrame()
for i in df['Pub_id'].unique().tolist():
        filtre_inter_pub_id = (df['Pub_id'] == i)
        df_inter = df[filtre_inter_pub_id]
        authors_fullname_list = '; '.join(df_inter['Nom Prénom'].tolist())
        df_inter['Authors Fullname List'] = authors_fullname_list
        df_out = df_out.append(df_inter)

In [ ]:
df_out.set_index('Pub_id')